In [1]:
from openmm.app import *
from openmm import *
from openmm.unit import *
import numpy as np
import plotly.graph_objects as go

# --- Parámetros del cubo ---
box_nm = 3.0
boxSize = Vec3(box_nm, box_nm, box_nm) * nanometers
forcefield = ForceField('amber14-all.xml', 'amber14/tip3p.xml')
modeller = Modeller(Topology(), [])

# --- Añadir agua y sal ---
modeller.addSolvent(forcefield,
                    model='tip3p',
                    boxSize=boxSize,
                    ionicStrength=5.0*molar,
                    positiveIon='Na+',
                    negativeIon='Cl-'
                    )

# --- CENTRAR el sistema en el cubo ---
positions = modeller.getPositions()
positions_array = np.array([[p.x, p.y, p.z] for p in positions])
# (En este punto, las posiciones ya están en nanómetros)

# Calcular centroide
min_xyz = positions_array.min(axis=0)
max_xyz = positions_array.max(axis=0)
center_system = (min_xyz + max_xyz) / 2
center_box = np.array([box_nm/2, box_nm/2, box_nm/2])
shift = center_box - center_system

# Trasladar todas las posiciones para centrar el sistema
new_positions = []
for p in positions_array:
    p_shifted = p + shift
    new_positions.append(Vec3(*p_shifted) * nanometer)
modeller.positions = new_positions

# --- Visualización estética ---
positions_array = np.array([[p.x, p.y, p.z] for p in modeller.positions])

elements = [atom.element.symbol for atom in modeller.topology.atoms()]
indices_O  = [i for i, e in enumerate(elements) if e == 'O']
indices_H  = [i for i, e in enumerate(elements) if e == 'H']
indices_Na = [i for i, e in enumerate(elements) if e == 'Na']
indices_Cl = [i for i, e in enumerate(elements) if e == 'Cl']

ox  = positions_array[indices_O]
hx  = positions_array[indices_H]
nax = positions_array[indices_Na]
clx = positions_array[indices_Cl]

# Líneas del cubo para visualización
cube_lines = []
for s, e in [([0,0,0],[box_nm,0,0]), ([0,0,0],[0,box_nm,0]), ([0,0,0],[0,0,box_nm]),
             ([box_nm,0,0],[box_nm,box_nm,0]), ([box_nm,0,0],[box_nm,0,box_nm]),
             ([0,box_nm,0],[box_nm,box_nm,0]), ([0,box_nm,0],[0,box_nm,box_nm]),
             ([0,0,box_nm],[box_nm,0,box_nm]), ([0,0,box_nm],[0,box_nm,box_nm]),
             ([box_nm,box_nm,0],[box_nm,box_nm,box_nm]), ([box_nm,0,box_nm],[box_nm,box_nm,box_nm]),
             ([0,box_nm,box_nm],[box_nm,box_nm,box_nm])]:
    cube_lines.append(go.Scatter3d(
        x=[s[0],e[0]], y=[s[1],e[1]], z=[s[2],e[2]],
        mode='lines',
        line=dict(color='white', width=4),
        showlegend=False
    ))

fig = go.Figure()

# Agua: 
fig.add_trace(go.Scatter3d(
    x=ox[:,0], y=ox[:,1], z=ox[:,2],
    mode='markers',
    marker=dict(size=3.5, color='red', opacity=0.7, line=dict(width=0.5, color='navy')),
    name='Oxígeno (O)',
    hoverinfo='skip'
))
fig.add_trace(go.Scatter3d(
    x=hx[:,0], y=hx[:,1], z=hx[:,2],
    mode='markers',
    marker=dict(size=2, color='white', opacity=0.65),
    name='Hidrógeno (H)',
    hoverinfo='skip'
))

# Na⁺:
if len(nax):
    fig.add_trace(go.Scatter3d(
        x=nax[:,0], y=nax[:,1], z=nax[:,2],
        mode='markers',
        marker=dict(size=7, color='royalblue', opacity=0.95, symbol='circle', line=dict(width=1.2, color='aqua')),
        name='Sodio (Na⁺)'
    ))

# Cl⁻: verde brillante, semitransparente
if len(clx):
    fig.add_trace(go.Scatter3d(
        x=clx[:,0], y=clx[:,1], z=clx[:,2],
        mode='markers',
        marker=dict(size=7, color='seagreen', opacity=0.95, symbol='circle', line=dict(width=1.2, color='lawngreen')),
        name='Cloruro (Cl⁻)'
    ))

# Añadir líneas del cubo
for line in cube_lines:
    fig.add_trace(line)

# Estética PRO
fig.update_layout(
    title='<b>Cubo de Agua y Sal</b>',
    scene=dict(
        xaxis=dict(title='X (nm)', range=[0, box_nm], showbackground=True, backgroundcolor='#222', gridcolor='#555', zerolinecolor='#999', showspikes=False),
        yaxis=dict(title='Y (nm)', range=[0, box_nm], showbackground=True, backgroundcolor='#222', gridcolor='#555', zerolinecolor='#999', showspikes=False),
        zaxis=dict(title='Z (nm)', range=[0, box_nm], showbackground=True, backgroundcolor='#222', gridcolor='#555', zerolinecolor='#999', showspikes=False),
        aspectmode='cube',
        camera=dict(eye=dict(x=1.4, y=1.45, z=1.1))
    ),
    paper_bgcolor='#111',
    plot_bgcolor='#111',
    font=dict(color='white', size=14),
    legend=dict(bgcolor='rgba(10,10,10,0.7)', bordercolor='white', borderwidth=1, x=0.02, y=0.98),
    margin=dict(l=0, r=0, b=0, t=60)
)

fig.show()
